<a href="https://colab.research.google.com/github/maggiemcc02/Failed_Adaptive_Meshing/blob/main/Newton_Meshes/Compare_Approx.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction
This notebook is the sister notebook to *Compare_Exact.* The idea is to have each notebook open next to each other so that you can compare the outputs. In particular, I will print the steps of the first iteration for both methods so that we can compare their output. 

In this notebook, I will print the first iteration of my approximated mesh solver (uses approximated $M$ values).

#### By: Maggie McCarthy, June 2022

In [ ]:
# import needed tools
import math
import numpy as np
from scipy.linalg import solve_banded
from numpy.linalg import norm
import matplotlib.pyplot as plt
from scipy.interpolate import interp1d
from scipy.optimize import fsolve

## Import the parts of the Approximate Solver
- physical solver
- mesh density function
- system
- jacobian
- jacobian approximation function
- solver

In [ ]:
def linear_nonuniform_bvp(f, a, c, grid, u_0, u_n):
  """
  f : right-hand side of ODE : a(x)u''(x) + c(x)u(x) = f(x)
  a : a(x) in ODE
  c : c(x) in ODE
  grid : the whole mesh
  u_0, u_n : boundary conditions
  
  """

  # take the interval ends from the mesh 
  # this is because we won't approximate at these points

  mesh = []
  for i in range(len(grid)):
    if i != 0:
      if i != (len(grid)-1):
        mesh.append([ grid[i][0] ])
  mesh = np.array(mesh)
  

  # compute m as the number of grid points we will approximate at

  m = len(mesh)

  # compute the mesh spacing (h)
  # associated with each grid point we will approximate at
  # hi = xi - xi-1

  h_list = []
  for i in range(1, len(grid)):
      h = grid[i][0] - grid[i-1][0]
      h_list.append( [h] )
  mesh_space = np.array(h_list)
  

  # 1. Set up our matrix A and our vector F

  upper = [] # list to save upper diagonal entries
  main = [] # list to save main diagonal entries
  lower = [] # list to save lower diagonal entries
  f_list = [] # list to save F entries

  for i in range(m):

    # retrieve x and h 
    # x is ith mesh entry
    # h is ith h entry 

    x = mesh[i][0]
    h1 = mesh_space[i][0]
    h2 = mesh_space[i+1][0]
                    
    

    if (i == 0): # first row of matrix


      # we add to only the main and upper diagonals

      upper_val = ( (a(x) * h1 ) / ( (1/2) * (h1 * (h2 **2) + h2 * (h1**2) ) ) )

      main_val = ( ((-1*(h1 + h2)*a(x) ) / ( (1/2) * (h1 * (h2 **2) + h2 * (h1**2) ) )) \
                   + c(x) )
                  

    
      upper.append(upper_val)
      main.append(main_val)

      # first and special entry of F vector

      f_entry = f(x) - ( (a(x)*h2) / ( (1/2) * (h1*(h2**2) + h2 * (h1**2)) ) ) * u_0

   
    elif (i == m-1): # last row of matrix

      # we add to only the lower and main diagonals

      main_val =   ( ((-1*(h1 + h2)*a(x) ) / ( (1/2) * (h1 * (h2 **2) + h2 * (h1**2) ) )) \
                   + c(x) )
      
      lower_val =  ( (a(x) * h2) / ( (1/2) * (h1 * (h2**2) + h2 * (h1**2)) ) )

      main.append(main_val)
      lower.append(lower_val)

      # last and special entry of F vector

      f_entry = f(x) - ( (a(x)*h1) / ( (1/2)* ( h1*(h2**2) + h2 * (h1**2) ) ) ) * u_n
 
    
    else: # general row

      # we add to all three diagonals

      upper_val = ( (a(x) * h1 ) / ( (1/2) * (h1 * (h2 **2) + h2 * (h1**2) ) ) )

      main_val =   ( ((-1*(h1 + h2)*a(x) ) / ( (1/2) * (h1 * (h2 **2) + h2 * (h1**2) ) )) \
                   + c(x) )
      
      lower_val = ( (a(x) * h2) / ( (1/2) * (h1 * (h2**2) + h2 * (h1**2)) ) )

    
      upper.append(upper_val)
      main.append(main_val)
      lower.append(lower_val)

      # general entry of F vector 

      f_entry = f(x)
  
    # append f_list with F vector entry

    f_list.append([f_entry])


  # create F vector 

  F = np.array(f_list)
    


  # create our banded matrix with 
  # row 1 = upper, row 2 = main, row 3 = lower
  A = np.array([ [0] + upper, main, lower + [0]])

 

  # solve the system AU = F using scipy's solve_banded

  U = solve_banded( (1, 1), A, F)


  # add the endpoints back in 

  solutions_list = [[u_0]]
  for i in U:
    solutions_list.append(i)
  solutions_list.append([u_n])
  solutions = np.array(solutions_list)


  return solutions

In [ ]:
# Mesh Density Function

def M(Ua, Ub, xa, xb):
  """
  Ua : either U_i or U_i+1
  Ub : either U_i-1 or U_i
  xa : either x_i or x_i+1
  xb : either x_i-1 or x_i

  """

  approx = ( (Ua - Ub) / (xa - xb) )**2

  result = math.sqrt( 1 + approx )

  return result 

In [ ]:
def approx_G(x, U, M, h):

  """
  x : uniform grid from physical solver (array of guesses)
  U : array with solution function approximations from the physical solver
  M : our mesh density function
  """
  g_list = []

  for i in range(len(x) - 2): # because we stop when i is the index of the third last entry 


      x0 = x[i][0] # x_i-1
      x1 = x[i+1][0] # x_i
      x2 = x[i+2][0] # x_i+1

      U0 = U[i][0] # U_i-1
      U1 = U[i+1][0] # U_i
      U2 = U[i+2][0] # U_i+1


      value = ( ( x0 * (M(U1, U0, x1, x0 )) )\
               - (x1 * (M(U1, U0, x1, x0) + M(U2, U1, x2, x1))) \
               + ( x2 *(M(U2, U1, x2, x1)) ) )
      
      
 

      g_list.append( [ value ] )

      
  # create array 
  G =  (1/(h**2))*np.array(g_list)
  return G

In [ ]:
# Jacobian

def approx_J(x, U, M, h):

  """
  x : uniform grid from physical solver (array of guesses)
  U : array with solution function approximations from the physical solver
  M : our mesh density function
  """
  main  = []
  upper = []
  lower = []
 

  for i in range(len(x) - 2): # because we stop when i is the index of the third last entry 

    if i == 0: # first row
    
      x0 = x[i][0] # x_i-1
      x1 = x[i+1][0] # x_i
      x2 = x[i+2][0] # x_i+1

      U0 = U[i][0] # U_i-1
      U1 = U[i+1][0] # U_i
      U2 = U[i+2][0] # U_i+1


      
      main.append( (x0 * ( ((-1) * (U1 - U0)**2) / (((x1-x0)**3) * M(U1, U0, x1, x0)) ) ) \
              - ( M(U0, U1, x0, x1) - x1 *  ( ((U1 - U0)**2) / ( ((x1-x0)**3) * M(U1, U0, x1, x0) ) ) )\
              - ( M(U2, U1, x2, x1) + x1 * ( ((U2 - U1)**2) / ( ((x2-x1)**3) * M(U2, U1, x2, x1)))) \
              + ( x2 * ( ((U2 - U1)**2) / ( ((x2-x1)**3) * M(U2, U1, x2, x1))) ) )
    
      upper.append(0)
      
      upper.append( M(U2, U1, x2, x1) - x2 * ( ((U2-U1)**2) / ( ((x2 - x1)**3) * M(U2, U1, x2, x1) ) )\
                + x1 * ( ((U2 - U1)**2) / ( ((x2-x1)**3) * M(U2, U1, x2, x1) ) ) )

    elif i == (len(x)-3):

      x0 = x[i][0] # x_i-1
      x1 = x[i+1][0] # x_i
      x2 = x[i+2][0] # x_i+1

      U0 = U[i][0] # U_i-1
      U1 = U[i+1][0] # U_i
      U2 = U[i+2][0] # U_i+1


      main.append( (x0 * ( ((-1) * (U1 - U0)**2) / (((x1-x0)**3) * M(U1, U0, x1, x0)))) \
              - ( M(U0, U1, x0, x1) - x1 *  ( ((U1 - U0)**2) / ( ((x1-x0)**3) * M(U1, U0, x1, x0) ) ) )\
              - ( M(U2, U1, x2, x1) + x1 * ( ((U2 - U1)**2) / ( ((x2-x1)**3) * M(U2, U1, x2, x1)))) \
              + ( x2 * ( ((U2 - U1)**2) / ( ((x2-x1)**3) * M(U2, U1, x2, x1))) ) )
      
      lower.append( M(U1, U0, x1, x0) + x0 * ( ((U1-U0)**2) / ( ((x1 - x0)**3) * M(U1, U0, x1, x0) ) )\
                - x1 * ( ((U1 - U0)**2) / ( ((x1-x0)**3) * M(U1, U0, x1, x0) ) ) )
      
      lower.append(0)

    else: 

      x0 = x[i][0] # x_i-1
      x1 = x[i+1][0] # x_i
      x2 = x[i+2][0] # x_i+1

      U0 = U[i][0] # U_i-1
      U1 = U[i+1][0] # U_i
      U2 = U[i+2][0] # U_i+1

      main.append( (x0 * ( ((-1) * (U1 - U0)**2) / (((x1-x0)**3) * M(U1, U0, x1, x0)))) \
              - ( M(U0, U1, x0, x1) - x1 *  ( ((U1 - U0)**2) / ( ((x1-x0)**3) * M(U1, U0, x1, x0) ) ) )\
              - ( M(U2, U1, x2, x1) + x1 * ( ((U2 - U1)**2) / ( ((x2-x1)**3) * M(U2, U1, x2, x1)))) \
              + ( x2 * ( ((U2 - U1)**2) / ( ((x2-x1)**3) * M(U2, U1, x2, x1))) ) )

      lower.append( M(U1, U0, x1, x0) + x0 * ( ((U1-U0)**2) / ( ((x1 - x0)**3) * M(U1, U0, x1, x0) ) )\
                - x1 * ( ((U1 - U0)**2) / ( ((x1-x0)**3) * M(U1, U0, x1, x0) ) ) )
      
      upper.append( M(U2, U1, x2, x1) - x2 * ( ((U2-U1)**2) / ( ((x2 - x1)**3) * M(U2, U1, x2, x1) ) )\
                + x1 * ( ((U2 - U1)**2) / ( ((x2-x1)**3) * M(U2, U1, x2, x1) ) ) )
    

  # create banded array
  jacob =  (1/(h**2)) * np.array( [ upper, main, lower ])


  return jacob

In [ ]:
# jacobian approximation function

def Jacobian(f, approx, M, x, n, E, h):
  """
  f : our system
  x : points to evaluate jacobian at
  n : number of equations in f
  E : epsilon
  """
  
  # set empty jacobian
  J = np.zeros((n, n))

  
  for row in range(n): # for each equation in f

    for column in range(len(x)-2): # for each variable in each equation
      

      epsilon = [ 0 for i in range(n) ]
      
      epsilon[column] = E
 

      x_E = []
      for i in range(len(x)):
        if i== 0:
          x_E.append([ x[0][0] ] )
        elif i ==(len(x)-1):
          x_E.append([ x[-1][0] ] )
        else:
          x_E.append([ x[i][0] + epsilon[i-1] ] )

      x_E = np.array(x_E)



      special_val = f(x_E, approx, M, h) 
      val = f(x, approx, M, h)


      entry = ( special_val[row] - val[row] ) / E


      J[row][column] = entry

  return J

In [ ]:
# the mesh solver

# define mesh solver without interpolation

def mesh_solver(G, J, x0, U, M, tol, max_iter, h):
  """
  Approximates the solution to a specific nonlinear system
  Parameters
  ---------
  G : Vector function
      Is the system of nonlinear equations
  
  J : Jacobian Matrix of G

  x0 : grid from physical solver (initial guess)

  U : solution from physical solver

  M : mesh density function

  tol : number 
        Tolerance

  max_iter: integer
            Max number of iterations

  Returns
  ------
  An approximation for the solutions of G(x) = 0 where G is a nonlinear system.
  """


  xn = x0 # sets initial guesses as xn for the first iteration of the loop
  
  for n in range(1, max_iter):

    print(" Our inital guess (xn) (initial mesh) = ")
    print(xn)
    print()

    # compute b = -G(X)
    
    b = (-1) * G(xn, U, M, h)

    print("Our system (-G) = ")
    print(b)
    print()    


    # compute A = J(X)

    A = J(xn, U, M, h)

 
    print("The Method's Jacobian =")
    print(A)
    print()  
    print("The Actual Jacobian (from Jacobian Function) = ")
    print(Jacobian(G, U, M, xn, 3, 1e-4, h))
    print()  

    # solve the system with scipy's solve_banded
    # we can use this solver because A is tridiagonal

    delta = solve_banded((1,1), A, b)

    print('The delta (solution to J(delta) = -G)')
    print(delta)
    print()   

    # update the interior entries in x (not first and last entry)
    x_new = []
    for i in range(len(xn)):
      if i == 0:
        x_new.append([ xn[i][0] ] )
      elif (i == (len(xn)-1)):
        x_new.append([ xn[i][0] ])
      else:
        x_new.append([ xn[i][0] + delta[i-1][0] ] )


      
    x_new = np.array(x_new)

    print('The new x (the new mesh)')
    print(x_new)
    print()
    

    if norm(x_new - xn) < tol:

      # x_new is our solution

      return x_new

    xn = x_new

  print('Max iterations reached and/or sequence is diverging')
  return None


## Note
In these functions, I multiply the system and the jacobian by $\frac{1}{h^2}$ so that the matrices are comparable to the matrices computed by the exact solver (which we multiply by $\frac{1}{h^2}$)

## Test Problem
I will print just one iteration and use only 5 points because we just want to compare the first iteration of this method to the exact method

In [ ]:
# test problem 

def f(x):
  return math.exp(x)

def a(x):
  return -1 * ((0.1) **2)

def c(x):
  return 1 


u_0 = 0
u_n = 0

# initial mesh

grid1 = np.linspace(0,1,5)

grid = np.array( [ grid1 ]).T

h = grid[1][0] - grid[0][0]

# compute U

U = linear_nonuniform_bvp(f, a, c, grid, 0, 0)

In [ ]:
mesh_solver(approx_G, approx_J, grid, U, M, 1e-5, 2, h)

 Our inital guess (xn) (initial mesh) = 
[[0.  ]
 [0.25]
 [0.5 ]
 [0.75]
 [1.  ]]

Our system (-G) = 
[[ 10.99782538]
 [  3.08354024]
 [-24.03128116]]

The Method's Jacobian =
[[  0.           7.89224511  12.73458027]
 [-11.24179384 -20.62682538 -14.93715014]
 [  7.89224511  12.73458027   0.        ]]

The Actual Jacobian (from Jacobian Function) = 
[[-11.24123986   7.89343939   0.        ]
 [  7.8910506  -20.62708591  12.73551354]
 [  0.          12.73364652 -14.93765124]]

The delta (solution to J(delta) = -G)
[[0.62906504]
 [2.2895443 ]
 [3.56076403]]

The new x (the new mesh)
[[0.        ]
 [0.87906504]
 [2.7895443 ]
 [4.31076403]
 [1.        ]]

Max iterations reached and/or sequence is diverging
